# Hyperparameter Tuning for a Deep Learning Regression Problem Using Talos

First import the necessary moduldes, load and split the data.

In [2]:
import numpy as np
import pickle
import os
import shutil
from keras import Sequential
from keras import layers
from keras.layers import Dense, Dropout
from keras.optimizers import Adamax, Adam, Nadam, Adadelta, Adagrad
from keras.activations import relu, sigmoid
from keras.callbacks import EarlyStopping

import talos
from talos.utils import hidden_layers
from talos.model.normalizers import lr_normalizer

import warnings
warnings.filterwarnings('ignore')

path = "/home/ubuntu/G/"
with open(path + "01_Data", "rb") as datafile:
    data = pickle.load(datafile)
    
X_df_train = data[0]
y_df_train = data[1]
X_df_test = data[2]
y_df_test = data[3]
PredData = data[4]
PredData1_binary = data[6]
baseseed = data[8]

datafile.close()

from sklearn.model_selection import train_test_split
X_df_train2, X_df_val, y_df_train2, y_df_val = train_test_split(X_df_train, y_df_train, test_size=0.33,
                                                                  random_state=baseseed, shuffle=True)


Set the parameter space

In [3]:
p = {'optimizer': [Adamax, Nadam, Adagrad, Adadelta],
     'first_neuron': [5120, 1024, 512, 128],
     'batch_size': [100, 1000, 5000, 10000],
     'epochs': [45],
     'hidden_layers':[0, 1, 2, 3, 4],
     'kernel_initializer': ['uniform', 'normal'],
     'dropout': [0.0, 0.25, 0.5, 0.65, 2, 3, 4, 5, 10],
     'losses': ["mse"],
     'shapes': ['brick', 'triangle', 'funnel'],
     'activation': ['relu'],
     'lr': list(np.linspace(0, 1, 10))
    }

Create the model function. The model includes an eraly stopping and includes metrics and losses for a regression problem.

In [4]:
def fcmodel(X_train, y_train, x_val, y_val, params):
    
    model = Sequential()
    
    esr = 2
    callbacks = [EarlyStopping(monitor="val_mean_absolute_error", patience=esr, mode="min", 
                               min_delta = 0.001, baseline=0.1)]
    
    model.add(Dense(params['first_neuron'], input_dim=X_train.shape[1],
                    activation="relu",
                    kernel_initializer = params['kernel_initializer'] ))
    model.add(Dropout(params['dropout']))
    
    ## hidden layers
    hidden_layers(model, params, 1)
    
    model.add(Dense(1, activation="sigmoid",
                    kernel_initializer=params['kernel_initializer']))
    
    model.compile(loss=params["losses"], 
                  optimizer=params['optimizer'](lr_normalizer(params['lr'], 
                                                params['optimizer'])),
                  metrics=['mae'])
    
    history = model.fit(X_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        callbacks=callbacks,
                        verbose=2)
    return history, model

Set a name for the project and set the path for saving everything. Also setting the scan parameters. I chose 100 rounds, and the tree reduction method. Since it showed much better results than the random serach.
For demo purposes I only included 1 round below.

In [6]:
proj = "Test01"
path2 = path + "LeNA_2/TalosTest/" + proj + "/"

if os.path.exists(path2):
    shutil.rmtree(path2)

t = talos.Scan(x=X_df_train2.values,
            y=y_df_train2.values,
            model=fcmodel,
            params=p,
            x_val=X_df_val.values,
            y_val=y_df_val.values,
            seed=baseseed,
            experiment_name=proj,
            print_params=True,
            round_limit=1)

  0%|          | 0/1 [00:00<?, ?it/s]WARNING: Logging before flag parsing goes to stderr.
W1216 12:42:15.624506 140127182698240 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1216 12:42:15.642313 140127182698240 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1216 12:42:15.645228 140127182698240 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1216 12:42:15.660840 140127182698240 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs

{'activation': 'relu', 'batch_size': 100, 'dropout': 0.25, 'epochs': 45, 'first_neuron': 512, 'hidden_layers': 0, 'kernel_initializer': 'uniform', 'losses': 'mse', 'lr': 0.3333333333333333, 'optimizer': <class 'keras.optimizers.Adagrad'>, 'shapes': 'triangle'}


W1216 12:42:15.952292 140127182698240 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 597956 samples, validate on 294516 samples
Epoch 1/45
 - 16s - loss: 0.0190 - mean_absolute_error: 0.0849 - val_loss: 0.0172 - val_mean_absolute_error: 0.0780
Epoch 2/45
 - 15s - loss: 0.0167 - mean_absolute_error: 0.0772 - val_loss: 0.0162 - val_mean_absolute_error: 0.0744
Epoch 3/45
 - 15s - loss: 0.0159 - mean_absolute_error: 0.0748 - val_loss: 0.0155 - val_mean_absolute_error: 0.0733
Epoch 4/45
 - 15s - loss: 0.0154 - mean_absolute_error: 0.0733 - val_loss: 0.0151 - val_mean_absolute_error: 0.0717
Epoch 5/45
 - 15s - loss: 0.0151 - mean_absolute_error: 0.0722 - val_loss: 0.0148 - val_mean_absolute_error: 0.0710
Epoch 6/45
 - 15s - loss: 0.0148 - mean_absolute_error: 0.0713 - val_loss: 0.0145 - val_mean_absolute_error: 0.0707
Epoch 7/45
 - 15s - loss: 0.0145 - mean_absolute_error: 0.0706 - val_loss: 0.0143 - val_mean_absolute_error: 0.0697
Epoch 8/45
 - 15s - loss: 0.0144 - mean_absolute_error: 0.0701 - val_loss: 0.0141 - val_mean_absolute_error: 0.0690
Epoch 9/45
 - 15s - 

100%|██████████| 1/1 [02:58<00:00, 178.15s/it]


Create the analysis dataframe

In [8]:
anafilename = os.listdir(path2)[0]
anafile = path2 + anafilename
r = talos.Reporting(anafile)

and_df = r.data
anadf = r.data.sort_values("val_mean_absolute_error")

Deploy the project for later usage of the model. Then evaluatiob takes place. In the live version I got an mae of approx. 0.029, which is pretty good.

In [9]:
path3 = path + "LeNA_2/TalosTest/"

if os.path.exists(path3 + proj + "_dep-zip"):
    os.remove(path3 + proj + "_dep-zip")

td = talos.Deploy(t, proj + "_dep", metric="val_mean_absolute_error", asc=True)

path_dep = path + "LeNA_2/TalosTest/"
talosfileslist = os.listdir(path_dep)
talosfileslist = [x for x in talosfileslist if x.find(proj + "_dep.zip") != -1]
talosfile = talosfileslist[0]

tr = talos.Restore(talosfile)

e = talos.Evaluate(t)

a = e.evaluate(X_df_test.values, y_df_test.values, metric="mae",
           task="continuous", folds=5, model_id=anadf.val_mean_absolute_error.idxmin())

print(a)

Deploy package Test01_dep have been saved.
[0.06487381557437902, 0.0659965060772302, 0.06566398204058764, 0.06538066394994979, 0.06628905642157068]
